In [7]:
from sgp4.api import Satrec
from sgp4 import exporter
import pykep as pk
import numpy as np

class tle:
    """__init__(line1, line2)

    This User Defined Planet (UDPLA) represents a satellite orbiting the Earth and defined in the TLE format
    and propagated using the SGP4 propagator.

    .. note::
       The resulting ephemerides will be returned in SI units and in the True Equator Mean Equinox (TEME) reference frame

    """
    def __init__(self, line1, line2):
        import pykep as pk
        self.satellite = Satrec.twoline2rv(line1, line2)
        self.e = 0
        self.ref_epoch = pk.epoch(self.satellite.jdsatepoch + self.satellite.jdsatepochF, pk.epoch.julian_type.JD)
    def eph(self, mjd2000):
        jd = mjd2000 + 51544 + 2451544.5
        jd_i = int(jd)
        jd_fr = jd-jd_i
        self.e, r, v = self.satellite.sgp4(jd_i, jd_fr)
        return [[it*1000 for it in r], [it*1000 for it in v]]
    def eph_v(self, mjd2000s):
        jds = [mjd2000 + 51544 + 2451544.5 for mjd2000 in mjd2000s]
        jd_is = [int(item) for item in jds]
        jd_frs = [a-b for a,b in zip(jds,jd_is)]
        self.e, r, v = self.satellite.sgp4_array(np.array(jd_is), np.array(jd_frs))
        rv = np.hstack((r,v))
        return rv.reshape((-1, 6)) 
    def get_name(self):
        return self.satellite.satnum_str + " - SGP4"
    def get_extra_info(self):
        line1, line2 = exporter.export_tle(self.satellite)
        return "TLE line1: " + line1 + "\nTLE line2: " + line2 
    def get_mu_central_body(self):
        return pk.MU_EARTH

In [12]:
line1 = '1 25544U 98067A   19343.69339541  .00001764  00000-0  38792-4 0  9991'
line2 = '2 25544  51.6439 211.2001 0007417  17.6667  85.6398 15.50103472202482'
udpla = tle(line1, line2)
mjd2000s = [when%1000.2 for when in range(1000)]
satellite = Satrec.twoline2rv(line1, line2)


In [9]:
pla = pk.planet(udpla)

In [13]:
%%timeit
for mjd2000 in mjd2000s:
    r = udpla.eph(mjd2000)

2.94 ms ± 523 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit
for mjd2000 in mjd2000s:
    r = pla.eph(mjd2000)

4.33 ms ± 13.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%timeit
r = pla.eph_v(mjd2000s)

7.8 ms ± 400 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit
for mjd2000 in mjd2000s:
    jd = mjd2000 + 51544 + 2451544.5
    jd_i = int(jd)
    jd_fr = jd-jd_i
    e, r, v = satellite.sgp4(jd_i, jd_fr)


2.01 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
jds = [mjd2000 + 51544 + 2451544.5 for mjd2000 in mjd2000s]


In [21]:
%%timeit
jds = [mjd2000 + 51544 + 2451544.5 for mjd2000 in mjd2000s]
jd_is = [int(item) for item in jds]
jd_frs = [a-b for a,b in zip(jds, jd_is)]
e, r, v = satellite.sgp4_array(np.array(jd_is), np.array(jd_frs))


7.13 ms ± 30.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
